# Build a Crew to Tailor Job Applications

In [1]:
# Warning control
import os
import warnings
from dotenv import load_dotenv,find_dotenv

warnings.filterwarnings('ignore')
load_dotenv(find_dotenv())

True

In [2]:
from phoenix.otel import register

# configure the Phoenix tracer
tracer_provider = register(
  project_name="my-crewai-vertex-app", # Default is 'default'
  auto_instrument=True # Auto-instrument your app based on installed OI dependencies
)

Overriding of current TracerProvider is not allowed


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: my-crewai-vertex-app
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [3]:
# Run gcloud auth application-default login before running this script
# to set up the credentials for the Google Cloud SDK
# and the Google Cloud client libraries.
# SERPER API key is required for this example. 
# You can set it up in your environment variables or directly in the code.


from crewai import LLM
from google.auth import default

credentials, _ = default()

llm = LLM(
    model="vertex_ai/gemini-2.5-flash-preview-05-20",
    temperature=0.7,
    # credentials=credentials,
    vertex_project=os.getenv("GOOGLE_CLOUD_PROJECT_ID"),
    vertex_location=os.getenv("GOOGLE_CLOUD_PROJECT_REGION"),
)

llm.call("Hello there Gemini! How are you doing today?")


os.environ["OPENAI_API_KEY"] = os.environ["GROQ_API_KEY"] 
os.environ["OPENAI_API_BASE"] = os.environ["GROQ_API_BASE"] 
os.environ["OPENAI_MODEL_NAME"] = 'meta-llama/llama-4-scout-17b-16e-instruct'

- Import libraries, APIs and LLM

In [4]:
from crewai import Agent, Task, Crew

## crewAI Tools

In [5]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

/home/padmanabhan/Documents/Github/Agents-Lab/.venv/lib/python3.12/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]


- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [6]:
from IPython.display import Markdown, display
display(Markdown("./fake_resume.md"))

# Padmanabhan  
- Email: padmanabhan@example.com  
- Phone: +91 98765 43210  

## Profile  
Padmanabhan is a dynamic AI & Data Science Manager with over a decade of experience delivering business-impacting solutions across data engineering, machine learning, and analytics. With a Master’s degree in Computer Science (Big Data and Data Science specialization) from Simon Fraser University, Canada, he brings a unique blend of academic rigor and hands-on industry expertise.

He currently leads a cross-functional team of data professionals and a UI designer at Ford Motor Company, developing scalable AI solutions with a strong focus on Generative AI applications, automation, and intelligent dashboarding. His work spans predictive modeling, data pipelines, and decision-support systems aligned with business strategy.

Skilled in Python, R, SQL, and modern data tools such as Tableau, Power BI, TensorFlow, and PySpark, Padmanabhan is known for building robust end-to-end pipelines, synthesizing actionable insights, and leading high-impact data initiatives. He is proactive, people-oriented, and passionate about continuous learning and innovation in AI.

## Work History  

### Ford Motor Company — Chennai, India (Hybrid)  
**Technical Project Manager – AI/ML**  
*Oct 2022 – Present*  
- Leading end-to-end development of AI/ML applications and dashboards.  
- Focused on business-driven Generative AI use cases and automation solutions.  
- Managing a team of data scientists, engineers, and a UI/UX designer.  

**Lead Engineer – AI/ML**  
*Feb 2021 – Oct 2022*  
- Developed predictive models and time series analysis tools.  
- Built crash data analysis solutions and synthetic data generation models.  
- Created interactive dashboards to support engineering decisions.  

### Simon Fraser University — Burnaby, Canada  

**Big Data Analyst**  
*May 2020 – Sep 2020*  
- Delivered AI/ML solutions for clients via SFU Big Data Hub.  
- Developed object detection models in TensorFlow Lite.  
- Created client dashboards using Tableau and Python.  

**Big Data Analyst / Research Assistant**  
*Jan 2019 – May 2020*  
- Built data pipelines and deployed ML models using Sklearn and TensorFlow.  
- Developed data visualizations with Tableau and Plotly.  

**Graduate Teaching Assistant**  
*Jan 2019 – Apr 2020*  
- TA for courses like Big Data Programming and Database Systems.  
- Supported students with coursework and exam preparation.  

### Avcorp Industries — Vancouver, Canada  
**Data Analyst Intern (Co-op)**  
*May 2019 – Aug 2019*  
- Built data ingestion and cleaning pipelines using Python and SQL.  
- Designed ML models to detect equipment failures.  
- Partnered with engineers to enhance manufacturing data use.  

### Cognizant Technology Solutions — Chennai, India  
**Associate – DWBI & Analytics**  
*Sep 2012 – Jul 2018*  
- Worked with major clients like JPMorgan Chase & Co., American Express, and H-E-B.  
- Built ETL pipelines using Informatica, performed data modeling and integration.  
- Developed BI dashboards with QlikView and Tableau.  
- Won several internal awards for performance and innovation.  

## Education  

**Master of Science in Computer Science (Big Data & Data Science)**  
Simon Fraser University, Canada  

**Bachelor’s Degree (Engineering)**  
Anna University, Chennai, India

## Skills  
- Programming: Python, R, SQL, PySpark  
- ML & AI: Scikit-learn, TensorFlow, Generative AI, Time Series Modeling  
- Tools: Tableau, Power BI, QlikView, TensorFlow Lite, Informatica  
- Data Engineering: ETL, Data Pipelines, Data Modeling  
- Soft Skills: Project Management, Team Leadership, Stakeholder Communication  


## Creating Agents

In [ ]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
    llm=llm
)

In [ ]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
    llm=llm
)

In [ ]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    ),
    llm=llm
)

In [ ]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    ),
    llm=llm
)

## Creating Tasks

In [ ]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [ ]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [ ]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [ ]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [ ]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [ ]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

- Dislplay the generated `tailored_resume.md` file.

In [ ]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

- Dislplay the generated `interview_materials.md` file.

In [ ]:
display(Markdown("./interview_materials.md"))

# CONGRATULATIONS!!!

## Share your accomplishment!
- Once you finish watching all the videos, you will see the "In progress" image on the bottom left turn into "Accomplished".
- Click on "Accomplished" to view the course completion page with your name on it.
- Take a screenshot and share on LinkedIn, X (Twitter), or Facebook.  
- **Tag @Joāo (Joe) Moura, @crewAI, and @DeepLearning.AI, (and a few of your friends if you'd like them to try out the course)**
- **Joāo and DeepLearning.AI will "like"/reshare/comment on your post!**

## Get a completion badge that you can add to your LinkedIn profile!
- Go to [learn.crewai.com](https://learn.crewai.com).
- Upload your screenshot of your course completion page.
- You'll get a badge from CrewAI that you can share!

(Joāo will also talk about this in the last video of the course.)